In [ ]:
import tensorflow as tf
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
plt.rc('font', size=16)
from sklearn.preprocessing import MinMaxScaler
import warnings
!pip install attention
from attention import Attention
warnings.filterwarnings('ignore')
tf.get_logger().setLevel('ERROR')

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)


#random seed
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [ ]:
#Import the dataset
dataset = pd.read_csv('../input/d/rossanalocatelli/training/Training.csv')
print(dataset.shape)

#To convert into float32
dataset = dataset.astype('float32')

dataset.head()
dataset.info()

In [ ]:
X_train_raw = dataset.iloc[:]
print(X_train_raw.shape)

# Normalize both features and labels
X_min = X_train_raw.min()
X_max = X_train_raw.max()

X_train_raw = (X_train_raw-X_min)/(X_max-X_min)

**Model for the first label 'Sponginess'**


In [ ]:
#Deciding window and stride 
window = 200
stride = 5

In [ ]:
future = dataset[-window:]
future = (future-X_min)/(X_max-X_min)
future = np.expand_dims(future, axis=0)
future.shape

In [ ]:
def build_sequences(df, target_labels=['Sponginess'], window=window, stride=stride, telescope=864): 
    assert window % stride == 0
    dataset = []
    labels = []
    temp_df = df.copy().values
    temp_label = df[target_labels].copy().values
    padding_len = len(df)%window

    if(padding_len != 0):
        padding_len = window - len(df)%window
        padding = np.zeros((padding_len,temp_df.shape[1]), dtype='float32')
        temp_df = np.concatenate((padding,df))
        padding = np.zeros((padding_len,temp_label.shape[1]), dtype='float32')
        temp_label = np.concatenate((padding,temp_label))
        assert len(temp_df) % window == 0

    for idx in np.arange(0,len(temp_df)-window-telescope,stride):
        dataset.append(temp_df[idx:idx+window])
        labels.append(temp_label[idx+window:idx+window+telescope])

    dataset = np.array(dataset)
    labels = np.array(labels)
    return dataset, labels

In [ ]:
target_labels = dataset.columns
telescope = 8

In [ ]:
X_train, y_train = build_sequences(X_train_raw, target_labels, window, stride, telescope)
X_train.shape, y_train.shape

In [ ]:
input_shape = X_train.shape[1:]
output_shape = y_train.shape[1:]
batch_size = 64
epochs = 500

In [ ]:
def build_CONV_LSTM_model(input_shape, output_shape):
    # Build the neural network layer by layer
    input_layer = tfkl.Input(shape=input_shape, name='Input')
    
    convlstm = tfkl.LSTM(128, return_sequences=True, kernel_initializer = tfk.initializers.GlorotUniform(seed))(input_layer) 
    convlstm = tfkl.LSTM(128, return_sequences=True,  kernel_initializer = tfk.initializers.GlorotUniform(seed))(convlstm)
    x1 = tfkl.Conv1D(256, 3, padding='same', activation='relu',  kernel_initializer = tfk.initializers.GlorotUniform(seed))(convlstm)
    convlstm = tfkl.GRU(256, return_sequences=True,  kernel_initializer = tfk.initializers.GlorotUniform(seed))(x1)
    x2 = tfkl.Conv1D(256, 3, padding='same', activation='relu',  kernel_initializer = tfk.initializers.GlorotUniform(seed))(convlstm) 
    added = tf.keras.layers.Add()([x1, x2]) #skip connection
    convlstm = tfkl.Conv1D(512, 3, padding='same', activation='relu', kernel_initializer = tfk.initializers.GlorotUniform(seed))(added)
    convlstm = tfkl.GlobalAveragePooling1D()(convlstm)
    convlstm = tfkl.Dropout(.5, seed=seed)(convlstm)

    dense = tfkl.Dense(output_shape[-1]*output_shape[-2], activation='relu',  kernel_initializer = tfk.initializers.GlorotUniform(seed))(convlstm)
    output_layer = tfkl.Reshape((output_shape[-2],output_shape[-1]))(dense)
    output_layer = tfkl.Conv1D(output_shape[-1], 1, padding='same',  kernel_initializer = tfk.initializers.GlorotUniform(seed))(output_layer)

    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

    model.compile(loss=tfk.losses.MeanSquaredError(), optimizer=tfk.optimizers.Adam(1e-5), metrics=['mae'])

    return model

In [ ]:
model = build_CONV_LSTM_model(input_shape, output_shape)
model.summary()
tfk.utils.plot_model(model, expand_nested=True)

In [ ]:
# Train the model
history = model.fit(
    x = X_train,
    y = y_train,
    batch_size = batch_size,
    epochs = epochs,
    validation_split=.1,
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=10, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=5, factor=0.5, min_lr=1e-5)
    ]
).history

In [ ]:
best_epoch = np.argmin(history['val_loss'])
plt.figure(figsize=(17,4))
plt.plot(history['loss'], label='Training loss', alpha=.8, color='#ff7f0e')
plt.plot(history['val_loss'], label='Validation loss', alpha=.9, color='#5a9aa5')
plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
plt.title('Mean Squared Error (Loss)')
plt.legend()
plt.grid(alpha=.3)
plt.show()

plt.figure(figsize=(17,4))
plt.plot(history['mae'], label='Training accuracy', alpha=.8, color='#ff7f0e')
plt.plot(history['val_mae'], label='Validation accuracy', alpha=.9, color='#5a9aa5')
plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
plt.title('Mean Absolute Error')
plt.legend()
plt.grid(alpha=.3)
plt.show()

plt.figure(figsize=(18,3))
plt.plot(history['lr'], label='Learning Rate', alpha=.8, color='#ff7f0e')
plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
plt.legend()
plt.grid(alpha=.3)
plt.show()

In [ ]:
model.save('SubmissionModel_0')

**Model for the second label 'Wonder Level'**

In [ ]:
window = 200
stride = 5

In [ ]:
future = dataset[-window:]
future = (future-X_min)/(X_max-X_min)
future = np.expand_dims(future, axis=0) 
future.shape

In [ ]:
target_labels = dataset.columns
telescope = 8

In [ ]:
X_train, y_train = build_sequences(X_train_raw, target_labels, window, stride, telescope)
X_train.shape, y_train.shape

In [ ]:
input_shape = X_train.shape[1:]
output_shape = y_train.shape[1:]
batch_size = 64
epochs = 500


In [ ]:
def build_CONV_LSTM_model(input_shape, output_shape):
    # Build the neural network layer by layer
    input_layer = tfkl.Input(shape=input_shape, name='Input')
    convlstm = tfkl.LSTM(128, return_sequences=True, kernel_initializer = tfk.initializers.GlorotUniform(seed))(input_layer) 
    convlstm = tfkl.LSTM(128, return_sequences=True, kernel_initializer = tfk.initializers.GlorotUniform(seed))(convlstm)
    x1 = tfkl.Conv1D(256, 3, padding='same', activation='relu',  kernel_initializer = tfk.initializers.GlorotUniform(seed))(convlstm)
    convlstm = tfkl.GRU(256, return_sequences=True,  kernel_initializer = tfk.initializers.GlorotUniform(seed))(x1)
    x2 = tfkl.Conv1D(256, 3, padding='same', activation='relu', kernel_initializer = tfk.initializers.GlorotUniform(seed))(convlstm)
    added = tf.keras.layers.Add()([x1, x2])
    convlstm = tfkl.Conv1D(512, 3, padding='same', activation='relu', kernel_initializer = tfk.initializers.GlorotUniform(seed))(added)
    convlstm = tfkl.GlobalAveragePooling1D()(convlstm)
    convlstm = tfkl.Dropout(.5, seed=seed)(convlstm)

    dense = tfkl.Dense(output_shape[-1]*output_shape[-2], activation='relu', kernel_initializer = tfk.initializers.GlorotUniform(seed))(convlstm)
    output_layer = tfkl.Reshape((output_shape[-2],output_shape[-1]))(dense)
    output_layer = tfkl.Conv1D(output_shape[-1], 1, padding='same', kernel_initializer = tfk.initializers.GlorotUniform(seed))(output_layer)

    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

    model.compile(loss=tfk.losses.MeanSquaredError(), optimizer=tfk.optimizers.Adam(1e-4), metrics=['mae'])

    return model

In [ ]:
model = build_CONV_LSTM_model(input_shape, output_shape)
model.summary()
tfk.utils.plot_model(model, expand_nested=True)

In [ ]:
# Train the model
history = model.fit(
    x = X_train,
    y = y_train,
    batch_size = batch_size,
    epochs = epochs,
    validation_split=.1,
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=10, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=5, factor=0.5, min_lr=1e-5)
    ]
).history

In [ ]:
best_epoch = np.argmin(history['val_loss'])
plt.figure(figsize=(17,4))
plt.plot(history['loss'], label='Training loss', alpha=.8, color='#ff7f0e')
plt.plot(history['val_loss'], label='Validation loss', alpha=.9, color='#5a9aa5')
plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
plt.title('Mean Squared Error (Loss)')
plt.legend()
plt.grid(alpha=.3)
plt.show()

plt.figure(figsize=(17,4))
plt.plot(history['mae'], label='Training accuracy', alpha=.8, color='#ff7f0e')
plt.plot(history['val_mae'], label='Validation accuracy', alpha=.9, color='#5a9aa5')
plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
plt.title('Mean Absolute Error')
plt.legend()
plt.grid(alpha=.3)
plt.show()


plt.figure(figsize=(18,3))
plt.plot(history['lr'], label='Learning Rate', alpha=.8, color='#ff7f0e')
plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
plt.legend()
plt.grid(alpha=.3)
plt.show()

In [ ]:
model.save('SubmissionModel_1')


**Model for the third and seventh label 'Crunchiness' and 'Hype Root'**

In [ ]:
window = 200
stride = 5

In [ ]:
future = dataset[-window:]
future = (future-X_min)/(X_max-X_min)
future = np.expand_dims(future, axis=0) 
future.shape

In [ ]:
target_labels = dataset.columns
telescope = 8

In [ ]:
X_train, y_train = build_sequences(X_train_raw, target_labels, window, stride, telescope)
X_train.shape, y_train.shape

In [ ]:
input_shape = X_train.shape[1:]
output_shape = y_train.shape[1:]
batch_size = 64
epochs = 500

In [ ]:
def build_CONV_LSTM_model(input_shape, output_shape):
    # Build the neural network layer by layer
    input_layer = tfkl.Input(shape=input_shape, name='Input')

    convlstm = tfkl.LSTM(128, return_sequences=True, kernel_initializer = tfk.initializers.GlorotUniform(seed))(input_layer)
    convlstm = tfkl.LSTM(128, return_sequences=True, kernel_initializer = tfk.initializers.GlorotUniform(seed))(convlstm)
    x1 = tfkl.Conv1D(256, 3, padding='same', activation='relu', kernel_initializer = tfk.initializers.GlorotUniform(seed))(convlstm)
    convlstm = tfkl.GRU(256, return_sequences=True, kernel_initializer = tfk.initializers.GlorotUniform(seed))(x1)
    x2 = tfkl.Conv1D(256, 3, padding='same', activation='relu', kernel_initializer = tfk.initializers.GlorotUniform(seed))(convlstm)
    added = tf.keras.layers.Add()([x1, x2])
    convlstm = tfkl.Conv1D(512, 3, padding='same', activation='relu', kernel_initializer = tfk.initializers.GlorotUniform(seed))(added)
    convlstm = tfkl.GlobalAveragePooling1D()(convlstm)
    convlstm = tfkl.Dropout(.5, seed=seed)(convlstm)

    dense = tfkl.Dense(output_shape[-1]*output_shape[-2], activation='relu', kernel_initializer = tfk.initializers.GlorotUniform(seed))(convlstm)
    output_layer = tfkl.Reshape((output_shape[-2],output_shape[-1]))(dense)
    output_layer = tfkl.Conv1D(output_shape[-1], 1, padding='same', kernel_initializer = tfk.initializers.GlorotUniform(seed))(output_layer)

    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

    model.compile(loss=tfk.losses.MeanSquaredError(), optimizer=tfk.optimizers.Adam(1e-4), metrics=['mae'])

    return model

In [ ]:
model = build_CONV_LSTM_model(input_shape, output_shape)
model.summary()
tfk.utils.plot_model(model, expand_nested=True)

In [ ]:
# Train the model
history = model.fit(
    x = X_train,
    y = y_train,
    batch_size = batch_size,
    epochs = epochs,
    validation_split=.1,
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=15, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=5, factor=0.5, min_lr=1e-5)
    ]
).history

In [ ]:
best_epoch = np.argmin(history['val_loss'])
plt.figure(figsize=(17,4))
plt.plot(history['loss'], label='Training loss', alpha=.8, color='#ff7f0e')
plt.plot(history['val_loss'], label='Validation loss', alpha=.9, color='#5a9aa5')
plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
plt.title('Mean Squared Error (Loss)')
plt.legend()
plt.grid(alpha=.3)
plt.show()

plt.figure(figsize=(17,4))
plt.plot(history['mae'], label='Training accuracy', alpha=.8, color='#ff7f0e')
plt.plot(history['val_mae'], label='Validation accuracy', alpha=.9, color='#5a9aa5')
plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
plt.title('Mean Absolute Error')
plt.legend()
plt.grid(alpha=.3)
plt.show()

plt.figure(figsize=(18,3))
plt.plot(history['lr'], label='Learning Rate', alpha=.8, color='#ff7f0e')
plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
plt.legend()
plt.grid(alpha=.3)
plt.show()

In [ ]:
model.save('SubmissionModel_2')

In [ ]:
model.save('SubmissionModel_6')

**Model for the fourth 'Loudness on impact'**

In [ ]:
window = 200
stride = 5

In [ ]:
future = dataset[-window:]
future = (future-X_min)/(X_max-X_min)
future = np.expand_dims(future, axis=0) 

In [ ]:
target_labels = dataset.columns
telescope = 8

In [ ]:
X_train, y_train = build_sequences(X_train_raw, target_labels, window, stride, telescope)
X_train.shape, y_train.shape

In [ ]:
input_shape = X_train.shape[1:]
output_shape = y_train.shape[1:]
batch_size = 64
epochs = 500

In [ ]:
def build_CONV_LSTM_model(input_shape, output_shape):
    # Build the neural network layer by layer
    input_layer = tfkl.Input(shape=input_shape, name='Input')

    convlstm = tfkl.LSTM(128, return_sequences=True, kernel_initializer = tfk.initializers.GlorotUniform(seed))(input_layer)
    convlstm = tfkl.LSTM(128, return_sequences=True, kernel_initializer = tfk.initializers.GlorotUniform(seed))(convlstm)
    x1 = tfkl.Conv1D(256, 3, padding='same', activation='relu', kernel_initializer = tfk.initializers.GlorotUniform(seed))(convlstm)
    x1 = tfkl.MaxPool1D(pool_size=4)(x1)
    convlstm = tfkl.GRU(256, return_sequences=True, kernel_initializer = tfk.initializers.GlorotUniform(seed))(x1)
    x2 = tfkl.Conv1D(256, 3, padding='same', activation='relu', kernel_initializer = tfk.initializers.GlorotUniform(seed))(convlstm)
    added = tf.keras.layers.Add()([x1, x2])
    convlstm = tfkl.Conv1D(512, 3, padding='same', activation='relu', kernel_initializer = tfk.initializers.GlorotUniform(seed))(added)
    convlstm = tfkl.GlobalAveragePooling1D()(convlstm)
    convlstm = tfkl.Dropout(.5, seed=seed)(convlstm)

    dense = tfkl.Dense(output_shape[-1]*output_shape[-2], activation='relu', kernel_initializer = tfk.initializers.GlorotUniform(seed))(convlstm)
    output_layer = tfkl.Reshape((output_shape[-2],output_shape[-1]))(dense)
    output_layer = tfkl.Conv1D(output_shape[-1], 1, padding='same', kernel_initializer = tfk.initializers.GlorotUniform(seed))(output_layer)

    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

    model.compile(loss=tfk.losses.MeanSquaredError(), optimizer=tfk.optimizers.Adam(1e-4), metrics=['mae'])

    return model

In [ ]:
model = build_CONV_LSTM_model(input_shape, output_shape)
model.summary()
tfk.utils.plot_model(model, expand_nested=True)

In [ ]:
# Train the model
history = model.fit(
    x = X_train,
    y = y_train,
    batch_size = batch_size,
    epochs = epochs,
    validation_split=.1,
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=10, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=5, factor=0.5, min_lr=1e-5)
    ]
).history

In [ ]:
best_epoch = np.argmin(history['val_loss'])
plt.figure(figsize=(17,4))
plt.plot(history['loss'], label='Training loss', alpha=.8, color='#ff7f0e')
plt.plot(history['val_loss'], label='Validation loss', alpha=.9, color='#5a9aa5')
plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
plt.title('Mean Squared Error (Loss)')
plt.legend()
plt.grid(alpha=.3)
plt.show()

plt.figure(figsize=(17,4))
plt.plot(history['mae'], label='Training accuracy', alpha=.8, color='#ff7f0e')
plt.plot(history['val_mae'], label='Validation accuracy', alpha=.9, color='#5a9aa5')
plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
plt.title('Mean Absolute Error')
plt.legend()
plt.grid(alpha=.3)
plt.show()

plt.figure(figsize=(18,3))
plt.plot(history['lr'], label='Learning Rate', alpha=.8, color='#ff7f0e')
plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
plt.legend()
plt.grid(alpha=.3)
plt.show()

In [ ]:
model.save('SubmissionModel_3')

**Model for the fifth label 'Meme creativity'**

In [ ]:
window = 200
stride = 10

In [ ]:
future = dataset[-window:]
future = (future-X_min)/(X_max-X_min)
future = np.expand_dims(future, axis=0)
future.shape

In [ ]:
target_labels = dataset.columns
telescope = 8

In [ ]:
X_train, y_train = build_sequences(X_train_raw, target_labels, window, stride, telescope)
X_train.shape, y_train.shape

In [ ]:
input_shape = X_train.shape[1:]
output_shape = y_train.shape[1:]
batch_size = 64
epochs = 400

In [ ]:
def build_CONV_LSTM_model(input_shape, output_shape):
    # Build the neural network layer by layer
    input_layer = tfkl.Input(shape=input_shape, name='Input')

    convlstm = tfkl.LSTM(64, return_sequences=True, kernel_initializer = tfk.initializers.GlorotUniform(seed))(input_layer)
    x1 = tfkl.Conv1D(128, 3, padding='same', activation='relu',  kernel_initializer = tfk.initializers.GlorotUniform(seed))(convlstm)
    convlstm = tfkl.GRU(128, return_sequences=True, kernel_initializer = tfk.initializers.GlorotUniform(seed))(x1)
    x2 = tfkl.Conv1D(128, 3, padding='same', activation='relu', kernel_initializer = tfk.initializers.GlorotUniform(seed))(convlstm)
    added = tf.keras.layers.Add()([x1, x2])
    convlstm = tfkl.Conv1D(256, 3, padding='same', activation='relu', kernel_initializer = tfk.initializers.GlorotUniform(seed))(added)
    convlstm = tfkl.Dropout(.3, seed=seed)(convlstm)
    convlstm = tfkl.LSTM(256, return_sequences=True, kernel_initializer = tfk.initializers.GlorotUniform(seed))(convlstm)
    convlstm = Attention()(convlstm)
    dense = tfkl.Dense(output_shape[-1]*output_shape[-2], activation='relu', kernel_initializer = tfk.initializers.GlorotUniform(seed))(convlstm)
    output_layer = tfkl.Reshape((output_shape[-2],output_shape[-1]))(dense)
    output_layer = tfkl.Conv1D(output_shape[-1], 1, padding='same', kernel_initializer = tfk.initializers.GlorotUniform(seed))(output_layer)

    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

    model.compile(loss=tfk.losses.MeanSquaredError(), optimizer=tfk.optimizers.Adam(1e-4), metrics=['mae'])

    return model

In [ ]:
model = build_CONV_LSTM_model(input_shape, output_shape)
model.summary()
tfk.utils.plot_model(model, expand_nested=True)

In [ ]:
# Train the model
history = model.fit(
    x = X_train,
    y = y_train,
    batch_size = batch_size,
    epochs = epochs,
    validation_split=.1,
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=10, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=5, factor=0.5, min_lr=1e-5)
    ]
).history

In [ ]:
best_epoch = np.argmin(history['val_loss'])
plt.figure(figsize=(17,4))
plt.plot(history['loss'], label='Training loss', alpha=.8, color='#ff7f0e')
plt.plot(history['val_loss'], label='Validation loss', alpha=.9, color='#5a9aa5')
plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
plt.title('Mean Squared Error (Loss)')
plt.legend()
plt.grid(alpha=.3)
plt.show()

plt.figure(figsize=(17,4))
plt.plot(history['mae'], label='Training accuracy', alpha=.8, color='#ff7f0e')
plt.plot(history['val_mae'], label='Validation accuracy', alpha=.9, color='#5a9aa5')
plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
plt.title('Mean Absolute Error')
plt.legend()
plt.grid(alpha=.3)
plt.show()

plt.figure(figsize=(18,3))
plt.plot(history['lr'], label='Learning Rate', alpha=.8, color='#ff7f0e')
plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
plt.legend()
plt.grid(alpha=.3)
plt.show()

In [ ]:
model.save('SubmissionModel_4')

**Model for the sixth label 'Soap slipperiness'**

In [ ]:
window = 150
stride = 5

In [ ]:
future = dataset[-window:]
future = (future-X_min)/(X_max-X_min)
future = np.expand_dims(future, axis=0)
future.shape

In [ ]:
target_labels = dataset.columns
telescope = 1

In [ ]:
X_train, y_train = build_sequences(X_train_raw, target_labels, window, stride, telescope)
X_train.shape, y_train.shape

In [ ]:
input_shape = X_train.shape[1:]
output_shape = y_train.shape[1:]
batch_size = 64
epochs = 500

In [ ]:
def build_CONV_LSTM_model(input_shape, output_shape):
    # Build the neural network layer by layer
    input_layer = tfkl.Input(shape=input_shape, name='Input')

    convlstm = tfkl.LSTM(128, return_sequences=True, kernel_initializer = tfk.initializers.GlorotUniform(seed))(input_layer)
    convlstm = tfkl.LSTM(128, return_sequences=True, kernel_initializer = tfk.initializers.GlorotUniform(seed))(convlstm)
    x1 = tfkl.Conv1D(256, 3, padding='same', activation='relu', kernel_initializer = tfk.initializers.GlorotUniform(seed))(convlstm)
    convlstm = tfkl.GRU(256, return_sequences=True, kernel_initializer = tfk.initializers.GlorotUniform(seed))(x1)
    x2 = tfkl.Conv1D(256, 3, padding='same', activation='relu', kernel_initializer = tfk.initializers.GlorotUniform(seed))(convlstm)
    added = tf.keras.layers.Add()([x1, x2])
    convlstm = tfkl.Conv1D(512, 3, padding='same', activation='relu', kernel_initializer = tfk.initializers.GlorotUniform(seed))(added)
    convlstm = tfkl.GlobalAveragePooling1D()(convlstm)
    convlstm = tfkl.Dropout(.5, seed=seed)(convlstm)

    dense = tfkl.Dense(output_shape[-1]*output_shape[-2], activation='relu', kernel_initializer = tfk.initializers.GlorotUniform(seed))(convlstm)
    output_layer = tfkl.Reshape((output_shape[-2],output_shape[-1]))(dense)
    output_layer = tfkl.Conv1D(output_shape[-1], 1, padding='same', kernel_initializer = tfk.initializers.GlorotUniform(seed))(output_layer)

    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

    model.compile(loss=tfk.losses.MeanSquaredError(), optimizer=tfk.optimizers.Adam(1e-4), metrics=['mae'])

    return model

In [ ]:
model = build_CONV_LSTM_model(input_shape, output_shape)
model.summary()
tfk.utils.plot_model(model, expand_nested=True)

In [ ]:
# Train the model
history = model.fit(
    x = X_train,
    y = y_train,
    batch_size = batch_size,
    epochs = epochs,
    validation_split=.1,
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=10, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=5, factor=0.5, min_lr=1e-5)
    ]
).history

In [ ]:
best_epoch = np.argmin(history['val_loss'])
plt.figure(figsize=(17,4))
plt.plot(history['loss'], label='Training loss', alpha=.8, color='#ff7f0e')
plt.plot(history['val_loss'], label='Validation loss', alpha=.9, color='#5a9aa5')
plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
plt.title('Mean Squared Error (Loss)')
plt.legend()
plt.grid(alpha=.3)
plt.show()

plt.figure(figsize=(17,4))
plt.plot(history['mae'], label='Training accuracy', alpha=.8, color='#ff7f0e')
plt.plot(history['val_mae'], label='Validation accuracy', alpha=.9, color='#5a9aa5')
plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
plt.title('Mean Absolute Error')
plt.legend()
plt.grid(alpha=.3)
plt.show()

plt.figure(figsize=(18,3))
plt.plot(history['lr'], label='Learning Rate', alpha=.8, color='#ff7f0e')
plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
plt.legend()
plt.grid(alpha=.3)
plt.show()

In [ ]:
model.save('SubmissionModel_5')